<a href="https://colab.research.google.com/github/songqsh/18ma573pub/blob/master/src/implied_vol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implied volatility

## Abstract 


- We will introduce implied volatility and its computations

# Code

In [0]:
'''======
run once, then comment it off, and run again
========'''
#!git clone https://github.com/songqsh/18ma573pub.git 
pass

Change working directory

In [4]:
cd 18ma573pub/src/

/content/18ma573pub/src


In [0]:
'''==========
here we import our own python modules
==================='''
import european_options_class as eoc
import sde_class as sde

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as so

In [18]:
'''===============
Test bsm_price
================='''
gbm1 = sde.Gbm(init_state = 100.,
                 drift_ratio = .0475,
                 vol_ratio = .2)
option1 = eoc.EuropeanOption(
    otype = 1, # 1: 'call'
              # -1: 'put'
    strike = 110.,                
    maturity = 1.
)
print('>>>>>>>call value is ' + str(gbm1.bsm_price(option1)))
option2 = eoc.EuropeanOption(otype=-1)
print('>>>>>>>>put value is ' + str(gbm1.bsm_price(option2)))


>>>>>>>call value is 5.943273183452838
>>>>>>>>put value is 10.84042522804176


In the market, usually volatility (vol = 0.2 in the above) is not available, but market price (call = 5.94) for the option is available. One wants find a volatility such that associated BSM formula is equal to (fit) the market price. 

In [0]:
'''==========
define a method to seek for an implied volatility
============'''

def implied_volatility(spot_price, otype, maturity, strike, interest_rate, market_option_price):
    option = eoc.EuropeanOption(otype, strike, maturity)
    init_vol = .1 #initial guess
    gbm = sde.Gbm(spot_price, interest_rate, init_vol)
    
    def error_function(vol):
        gbm.vol_ratio = vol
        return np.abs(market_option_price - gbm.bsm_price(option))
    
    return so.fmin(error_function, 0.1, disp = False)[0]

In [22]:
'''============
test the implied_vol by reversing bsm_formula example in the above
=============='''
print('>>>>>>>>implied volatility is')
implied_volatility(spot_price=100, otype=1, maturity=1, 
                   strike=110, interest_rate=.0475, market_option_price=5.94)

>>>>>>>>implied volatility is


0.19992187500000036